# This program connects to Box.com, gets all of the folders and images and stores information into the database.

https://developer.box.com/en/guides/authentication/oauth2/with-sdk/

In [1]:
!pip install boxsdk
!pip install auth
!pip install redis
!pip install mysql.connector
!pip install requests

  Using cached boxsdk-2.7.1-py2.py3-none-any.whl (126 kB)
  Using cached requests_toolbelt-0.9.1-py2.py3-none-any.whl (54 kB)
  Using cached requests-2.22.0-py2.py3-none-any.whl (57 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
     |████████████████████████████████| 125 kB 3.4 MB/s eta 0:00:01
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using cached auth-0.5.3-py3-none-any.whl (14 kB)
     |████████████████████████████████| 4.6 MB 3.5 MB/s eta 0:00:01     |█████████████████▋              | 2.5 MB 3.5 MB/s eta 0:00:01
  Using cached eventlet-0.25.1-py2.py3-none-any.whl (222 kB)
  Using cached gunicorn-20.0.4-py2.py3-none-any.whl (77 kB)
  Using cached mongoengine-0.19.1.tar.gz (157 kB)
  Using cached blinker-1.4.tar.gz (111 kB)
     |████████████████████████████████| 41 kB 274 kB/s  eta 0:00:01
  Using cached dnspython-1.16.0-py2.py3-none-any.whl (188 kB)
  Using cached monotonic-1.5-py2.py3-none-any.whl (5.3 kB)
     |████████████████████████████████| 460 k

# Run the cell below to authenticate with Box.com so we can use the API

In [1]:
from boxsdk import OAuth2

import json
#Set the file we want to use for authenticating a Box app
#The json file stores the client_id and client_secret so we don't have it in the code.
# The json file looks like this:
#{
#"client_id":"___the_codes_for_client_id___",
#"client_secret":"___the_codes_for_client_secret___"
#}

oauth_settings_file = '/home/student/jeff_not_data_5000/programs/box_app_test.json'
with open(oauth_settings_file, "r") as read_file:
    oauth_data = json.load(read_file)
print(oauth_data["client_id"])
print(oauth_data["client_secret"])

oauth = OAuth2(
    client_id=oauth_data["client_id"],
    client_secret=oauth_data["client_secret"]
)

auth_url, csrf_token = oauth.get_authorization_url('https://jeffblackadar.ca')
print("Click on this:")
print(auth_url)
print(csrf_token)
print("Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.")


rvy5o5ic72c00ldi8vpcqmof0t7h6w85
VWJQ1LTracitLw5HfG8Q0TKplBogFEIT
Click on this:
https://account.box.com/api/oauth2/authorize?state=box_csrf_token_bMTNEZACGRXj20bK&response_type=code&client_id=rvy5o5ic72c00ldi8vpcqmof0t7h6w85&redirect_uri=https%3A%2F%2Fjeffblackadar.ca
box_csrf_token_bMTNEZACGRXj20bK
Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.


In [2]:
from boxsdk import Client

# Make sure that the csrf token you get from the `state` parameter
# in the final redirect URI is the same token you get from the
# get_authorization_url method to protect against CSRF vulnerabilities.
#assert 'THE_CSRF_TOKEN_YOU_GOT' == csrf_token
access_token, refresh_token = oauth.authenticate('aubN94nFeXUNNO6zncOwqkUBHUi5Vv6U')
client = Client(oauth)

## Test the connection to Box.com - list all folders and files using recursion

In [ ]:
def process_subfolder_test(client, folder_id, folder_name):
    print("this folder: "+folder_name)
    items = client.folder(folder_id=folder_id).get_items()
    for item in items:
        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
        if(item.type.capitalize()=="Folder"):
            process_subfolder_test(client, item.id,folder_name+"/"+item.name)
        if(item.type.capitalize()=="File"):
            #print(item)
            print('File: {0} is named: "{1}" path: {2} '.format(item.id, item.name, folder_name+"/"+item.name))            
    return

process_subfolder_test(client, '0',"")

## Start MySQL

sudo systemctl start mysql

## Get all folders and store all files into a database.

In [17]:
def process_subfolder(client, folder_id, folder_name,cursor_insert):
    print("this folder: "+folder_name)
    items = client.folder(folder_id=folder_id).get_items()
    for item in items:
        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
        if(item.type.capitalize()=="Folder"):
            process_subfolder(client, item.id,folder_name+"/"+item.name,cursor_insert)
        if(item.type.capitalize()=="File"):
            #print(item)
            print('File: {0} is named: "{1}" path: {2} '.format(item.id, item.name, folder_name+"/"+item.name))
            
            insert_image = ("INSERT INTO tbl_box_images (`id_box_file`,`id_box_folder`,`folder`,`file_name`,`img_src`,`img_url`) VALUES (%s,%s,%s,%s,%s,%s);")
            
            cursor_insert.execute(insert_image, (item.id, folder_id, folder_name, item.name, folder_name+"/"+item.name, ""))
            mariadb_connection.commit()
    return





In [ ]:
import requests
import shutil
from pathlib import Path
base_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\pompeii\\box'


import mysql.connector as mariadb

mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")

cursor_insert = mariadb_connection.cursor(buffered=True)

process_subfolder(client, '0',"", cursor_insert)
    
mariadb_connection.close()    


In [20]:
mariadb_connection.close()    

## Process images: Connect to Box.com, download image, get its hash value.

In [6]:
 !pip install imutils

  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25853 sha256=60a616b9e2cabad44117bf93922a209f14a7f5a84c2998c601ce231cfe3d5d4c
  Stored in directory: /home/student/.cache/pip/wheels/27/4d/3a/6dcdf7c3ebc87bf1ae013d96c9cf060ccfe334bb5ee769f377
Successfully built imutils


In [3]:
# Image hash functions
# import the necessary packages
# Credit Adrian Rosebrock https://www.pyimagesearch.com
from imutils import paths
import argparse
import time
import sys
import cv2
import os

# hashing with OpenCV and PythonPython
def dhash(image, hashSize=8):
    # resize the input image, adding a single column (width) so we
    # can compute the horizontal gradient
    resized = cv2.resize(image, (hashSize + 1, hashSize))

    # compute the (relative) horizontal gradient between adjacent
    # column pixels
    diff = resized[:, 1:] > resized[:, :-1]

    # convert the difference image to a hash
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])


In [6]:
download_process_box_thumbnail(client,3063, "580416299144", "/pinp5/Maps/Maps", "Plan Bay of Naples 1865 from Karl von Spruner’s 1865 rendering of Southern Italy and Sicily in antiquity Wikimedia.jpg", cursor_update)

NameError: name 'cursor_update' is not defined

In [4]:
import os
import shutil
from pathlib import Path
base_dir = '/home/student/jeff_not_data_5000/images/box/'

# using the thumbnail for image hashing does not work   
def download_process_box_full_id_box_file(client, id_box, id_box_file, folder, file_name, cursor_update):
    #Some file names can't be processed.  This uses the box file name instead.
    print("this file: "+id_box_file+" "+file_name)
    #using the the large 320 pixel thumbnail produces different results than the full image - it does not work
    #try:
    #file_info = client.file(id_box_file).get()
    file_name = id_box_file+".jpg"
    imagecontent = client.file(id_box_file).content()
    # load the image for the image hash later.

    img_local_path=base_dir+folder+"/"+file_name
    #print(img_local_path)
    img_file_name = file_name
    img_local_folder = base_dir+folder

    #see if the image is there already
    if not os.path.exists(img_local_path):
        img_file_name = file_name
        img_local_folder = base_dir+folder
        
        if not os.path.exists(img_local_folder):
            #os.mkdir(img_local_folder)
            path = Path(img_local_folder)
            path.mkdir(parents=True,exist_ok=True)
            #print(img_local_folder+'/'+ img_file_name)
            
            # Open the url image, set stream to True, this will return the stream content.
            # Open a local file with wb ( write binary ) permission.
            #local_file = open(img_local_folder+'/'+ img_file_name, 'wb')
        with open(img_local_folder+'/'+ img_file_name,'wb') as f:
                f.write(imagecontent)
            #print(thumbnail)
        del imagecontent
        f.close()
            #image hash
    imageHash = 0
    #print("image hash")
    if not os.path.exists(img_local_path):
        print("Error image not loaded for image hash   " + img_local_path)
    else:         
        # load the image from disk
        path = Path(img_local_folder)
        os.chdir(path)
        hash_image = cv2.imread(img_file_name)
 
        # if the image is None then we could not load it from disk (so skip it)
        if not hash_image is None:
            # convert the image to grayscale and compute the hash
            hash_image = cv2.cvtColor(hash_image, cv2.COLOR_BGR2GRAY)
            imageHash = dhash(hash_image)
            #print(imageHash)
        else:
            imageHash = 0
            #print("image hash - can't find image?")
        update_image = ("UPDATE tbl_box_images SET image_hash = %s WHERE id = %s;")
        print(id_box, imageHash)
        cursor_update.execute(update_image, (imageHash, id_box))
        mariadb_connection.commit()
        
        
        os.remove(img_local_path)
    return
    #except:
    #    print(sys.exc_info()[0])
    #    return
    

In [ ]:
number_of_rows_to_run = 100000
start_id = 165806
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='/home/student/jeff_not_data_5000/programs/webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_update = mariadb_connection.cursor(buffered=True)
cursor.execute("SELECT id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src FROM webpage_images.tbl_box_images WHERE id>=%s AND id<%s", (start_id, end_id))
for id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src in cursor:
    #print(id, id_box_file, img_path_match, id_tbl_webpage_images, img_src)
    #don't include .xml, .htm* or .thmx
    #for row 158348
    if(img_path_match is None):
        img_path_match = ""
    if (img_src.lower().find(".xml")<0) and (img_src.lower().find(".htm")<0) and (img_src.lower().find(".thmx")<0) and (img_src.lower().find(".js")<0) and (img_src.lower().find(".pdf")<0) and (img_src.lower().find(".zip")<0) and (img_src.lower().find(".asp")<0) and (img_src.lower().find(".txt")<0) and (img_src.lower().find(".ico")<0):
        download_process_box_full_id_box_file(client, id, str(id_box_file), folder, file_name, cursor_update)
            
cursor.close()
cursor_update.close()
mariadb_connection.close()    


In [47]:
mariadb_connection.close()    